# <center>پیدا کردن ربات</center>

<div dir="rtl" style="font-size:18px">
احتمالا کارتون WALL-E را بخاطر دارید. رباتی که توانست زمین را نجات دهد و باقیمانده نسل انسان‌ها را به زمین برگرداند. پس از بازگشت انسان‌ها به زمین و پس از محبوبیت زیاد WALL-E، او دوستان زیادی پیدا کرد. بهترین دوست او WALL-F یکی از بازیگوش‌ترین ربات‌هایی است که تاحالا دیده‌اید.
<br>
یکی از دفعاتی که WALL-F به بیرون رفته بود، راهی به درون یک هزار‌تو میابد. تصویر زیر از او در این هزارتو گرفته شده است.
</div>
<br>
<br>
<center>

![WALL-F Ai-Generated](Assets/robot.jpg)

</center>

<div dir="rtl">
در ادامه میخواهیم با روش‌های مختلفی پیدا کنیم که در حال حاضر ربات در چه خانه‌ای قرار دارد. در هرکدام از تست‌کیس‌هایی که لازم است بررسی کنید. خانه 0 و 0 خانه بالا و چپ هزارتو خواهد بود. در ادامه برای هر خانه مشخص خواهد شد که دیواری به هرکدام از جهات دارد یا خیر. 
به طور مثال عبارت
''R L''
به این معنی است که این خانه فقط به چپ و راست خود راه دارد. همچنین شماره‌گذاری خانه‌ها به صورت زیر است.
در هر مرحله او به شما میگوید که در کدام خانه از جدول است. عددی که ربات به شما میگوید یک عدد تصادفی است که در یک ماتریس به شما داده میشود. عدد سطر 
$i$ 
و ستون 
$j$ 
احتمال این را نشان می‌دهد که ربات در خانه 
$i$ 
باشد اما خانه 
$j$ 
را شما گزارش دهد. این ربات در پایان هر مرحله با احتمال برابر به یکی از خانه‌های مجاور میرود. همینطور خانه شروع این ربات هم توزیعی یکنواخت دارد.
</div>
<br>
<br>
<center>

![grid](Assets/grid.jpg)

</center>

<div dir="rtl">
در ابتدا برای پیاده‌سازی ۲ الگوریتمی که در ادامه خواهید دید به آبجکتی از نوع Grid نیاز خواهید داشت. عملکرد این آبجکت را میتوانید از مسیر Assets/helpers.py ببینید. به طور مثال هزارتو زیر را در نظر بگیرید.
</div>

<center>

![maze](Assets/grid2.jpg)

</center>

<div dir="rtl">
grid متناظر با این هزارتو را میتوانید از طریق تابع get_sample_grid بگیرید و با توابع آن آشنا شوید. در صورت لزوم میتوانید هر تابع دلخواهی به این کلاس اضافه کنید. برای اطلاع از تعداد سطرهای grid میتوانید از grid.n و برای اطلاع از تعداد ستون‌ها میتوانید از grid.m استفاده کنید.
</div>

In [1]:
from Assets.helpers import get_sample_grid
import numpy as np

In [2]:
grid = get_sample_grid()
print(grid.n)
print(np.random.choice([1,2,3],size=10))
print(np.argsort([1,2,3,4]))


2
[3 3 1 2 3 3 2 2 3 1]
[0 1 2 3]


# Particle Filtering

<div dir="rtl">
در این قسمت باید آخرین خانه‌ای که این ربات در آن قرار دارد را با استفاده از الگوریتم Particle Filtering پیدا کنید. برای اینکار در ابتدا تعداد Particle هایی با تعدادی از اردر تعداد خانه‌های هزارتو بسازید. در پایان هر مرحله جمعیت را با توجه به وزن آنها نصف کنید و ذراتی جدید متناسب با تعداد ذرات درون هرخانه بسازید. در پایان خانه‌ای که بیشترین ذرات در آن قرار دارد را برگردانید.
</div>

In [3]:
# nesf nakonid!!!!!!!!!
num = 10
def move(particle,grid):
    c = np.random.choice(grid.moves[int(particle[0])][int(particle[1])])
    if c == 'U':
        particle[0] -= 1
    elif c == 'D':
        particle[0] += 1
    elif c == 'L':
        particle[1] -= 1
    elif c == 'R':
        particle[1] += 1    

def PF(grid, prob ,obs):
    """
    input:

    grid: object of type Grid
    obs: array of ints, each one is an observation

    output: 
    number of the last cell
    """
    particles = np.empty((num*grid.n * grid.m, 2)).astype(int)
    particles[:,0] = np.random.randint(0,grid.n, size=num* grid.n * grid.m)
    particles[:,1] = np.random.randint(0,grid.m, size=num* grid.n * grid.m)

    
    for i in range(len(obs)):
        for j in range(len(particles)):
            move(particles[j],grid)
        weigths = np.ones(num*(grid.n * grid.m))
        for j in range(len(weigths)):
            weigths[j] = prob[grid.m*particles[j][0] + particles[j][1],obs[i]] 

        # resample
        # comment this part out if you want better performance, 
        """"       
        args = np.argsort(weigths)[int((num*grid.n * grid.m)/2):]
        weigths = weigths[args]
        particles = particles[args]
        particles = np.append(particles,particles,axis=0)
        """
        
        
        weigths /= sum(weigths)
        new_particles = np.empty(((grid.n * grid.m) * num, 2)).astype(int)
        idx = [i for i in range(len(particles))] 
        idx = np.random.choice(idx, size=num*(grid.n * grid.m), p=weigths) 
        for j in range(len(idx)):
            new_particles[j] = particles[idx[j]]
        particles = new_particles 
        
    number_of_particle = np.zeros(num*(grid.n * grid.m))
    for j in particles:
        number_of_particle[grid.m*j[0] + j[1]] += 1
    a = number_of_particle.argmax()  
    #print(a, number_of_particle[a])    
    return a

In [4]:
from Assets.helpers import *
PF_checker(PF)


C:\Users\moeen\AppData\Local\Temp\ipykernel_4828\3699553482.py:24: RuntimeWarning: invalid value encountered in cast
  particles = np.empty((num*grid.n * grid.m, 2)).astype(int)
C:\Users\moeen\AppData\Local\Temp\ipykernel_4828\3699553482.py:47: RuntimeWarning: invalid value encountered in cast
  new_particles = np.empty(((grid.n * grid.m) * num, 2)).astype(int)


# HMM

<div dir="rtl">
در این قسمت غیر از اینکه باید بفهمیم WALL-F در کدام خانه قرار دارد. میخواهیم بدانیم که محتمل‌ترین مسیری که طی کرده‌ است چه مسیری است. برای اینکار الگوریتم viterbi را پیاده‌سازی کنید.
</div>

In [29]:
def tp(s,prev_s,grid):
    if s== prev_s+1:
        m="R"
    elif s== prev_s-1:
       m="L"
    elif s== prev_s-grid.m:
       m="U"
    elif s== prev_s + grid.m:
       m="D"
    else:
        return 0
    if m in grid.moves[int(prev_s/grid.m)][prev_s % grid.m]:
        return 1/len(grid.moves[int(prev_s/grid.m)][prev_s % grid.m])
    return 0
def viterbi(grid,prob ,obs):
    """
    input:

    grid: object of type Grid
    obs: array of ints, each one is an observation

    output: 
    array of cell numbers, same shape as obs
    """
    s_probs = [[0.0 for _ in range(grid.n * grid.m)] for _ in range(len(obs))]
    best_p_s = [[0 for _ in range(grid.n * grid.m)] for _ in range(len(obs))]
    for t in range(len(obs)):
        for s in range(grid.n * grid.m):
            if t == 0:
                s_probs[t][s] = 1/(grid.n * grid.m) * prob[s][obs[t]]
            else:
                best_p_s[t][s] = max(range((grid.n * grid.m)), key=lambda prev_s: s_probs[t-1][prev_s] * tp(s,prev_s,grid))
                max_prob = s_probs[t-1][best_p_s[t][s]] * tp(s,best_p_s[t][s],grid)
                s_probs[t][s] = max_prob * prob[s][obs[t]]

    best_path_pointer = max(range((grid.n * grid.m)), key=lambda s: s_probs[-1][s])
    MLE = [best_path_pointer]
    for t in range(len(obs)-1, 0, -1):
        MLE.insert(0, best_p_s[t][MLE[0]])
    return MLE

In [30]:
from Assets.helpers import viterbi_checker
viterbi_checker(viterbi)

KeyboardInterrupt: 